In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF
from sklearn.datasets import fetch_20newsgroups
from sklearn.metrics.pairwise import cosine_similarity as cosine
from scipy.sparse.csgraph import connected_components
import itertools
import json
import numpy as np
import glob
n_topics = 20
n_top_words = 20
###fake data
# dataset = fetch_20newsgroups(shuffle=True, random_state=1,
#                              remove=('headers', 'footers', 'quotes'))
# data_samples = dataset.data

In [10]:
from semi_supervised_nnmf import ssnmf, t

In [7]:
local_data = []
classes = ['afghannationalliberationfront', 'hezbislami', 'jamiatislami']
philes =  glob.glob("/Users/ziv/GDrive/school/math-thesis/nmf-imp/txt_data_bypage/*.txt")
Y = np.zeros((len(classes),len(philes)))
for (i,phile) in enumerate(philes):
    c = phile.split('/')[-1].split('_')[0]
    cls = classes.index(c)
    Y[cls,i]= 1
    with open(phile, 'r') as myfile:
        data=myfile.read().replace('\n', '')
        local_data.append(unicode(data, errors='ignore'))

In [34]:
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, #max_features=n_features,
                                   stop_words='english')

X = tfidf_vectorizer.fit_transform(local_data)
tfidf_feature_names = tfidf_vectorizer.get_feature_names()
k = len(classes)
r = 20
lamb = 1
L = np.ones((k,X.shape[0]))


In [35]:
(A,S,B) = ssnmf(t(X.toarray()),L,Y,lamb,r,k)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100


In [36]:
print X.shape
print A.shape
print S.shape

H = t(A)
W = t(S)

print X.shape
print W.shape
print H.shape



(1511, 15380)
(15380, 20)
(20, 1511)
(1511, 15380)
(1511, 20)
(20, 15380)


In [37]:
H = t(A)
W = t(S)

#x = WH
weights = (5000/W.sum())*W.sum(axis=0)

def build_wgraph(alpha=2):
    if alpha != 2:
        return [[int(cosine(H[i],H[j])[0][0] > alpha) for i in range(0, len(H))] for j in range(0, len(H))]
    else:
        return [[cosine(H[i],H[j])[0][0] for i in range(0, len(H))] for j in range(0, len(H))]
def thresh_vals(numbin):
    binz = []
    w = build_wgraph(2)
    chain = itertools.chain(*w)
    s =sorted(list(chain))
    val = n_topics*n_topics/numbin
    for i,v in enumerate(s):
        if i%val ==0: binz.append(v)
    return binz

In [92]:
def array_distance(A,B):
    count = 0
    for i,x in enumerate(A):
        if x == B[i]:
            count+=1
    return len(A)-count

def greedy_TV_build(to_consume,bins):
    if len(to_consume)>1:
        a = to_consume[0]
        b = to_consume[1]
        ccA = connected_components(build_wgraph(a))[1]
        ccB = connected_components(build_wgraph(b))[1]
        if not np.array_equal(ccA, ccB):
            distance = array_distance(ccA,ccB)
            if distance > 8:
                new_tv = [a + i*(b-a)/bins for i in range(0,bins)]
                return new_tv + greedy_TV_build( to_consume[1:], bins)
            else:
                return [a] + greedy_TV_build(to_consume[1:], bins)
        else:
            return greedy_TV_build(to_consume[1:], bins)
    elif len(to_consume) == 1:
        return to_consume
    else:
        return []
  

def populateTree(row_level, valid_community):
    if row_level > size-2 :
        return []
    else:
        children = []
        parent_community = connected_components(build_wgraph(tv[row_level]))[1]
        child_community = connected_components(build_wgraph(tv[row_level+1]))[1]
        unique_communities = list(set(parent_community)) 
        for unique_community in unique_communities:
            if valid_community == unique_community:
                indices = [i for i, x in enumerate(parent_community) if x == unique_community] #[8,9]
                seen_communities = []
                for i in indices: #8 and 9
                    if child_community[i] in seen_communities:
                        filter(lambda x: x['community'] == str(child_community[i]), children)[0]['indices'].append(i)
                    else:
                        community_to_find = child_community[i]
                        grow_my_children = populateTree(row_level+1, community_to_find)
                        if grow_my_children:
                            name = ""
                            color = sum(B)[i]
                            print (color,i)
                            children.append({"community":str(child_community[i]),"indices":[i],"name" : name,"color":color, "children":grow_my_children, "hasChildren": True})
                        else:
                            color = sum(B)[i]
                            print (color,i)
                            name = " ".join([tfidf_feature_names[j] for j in H[i].argsort()[:-n_top_words - 1:-1]])
                            children.append({"community":str(child_community[i]),"indices":[i],"size":weights[i],"name":name,"color":color, "hasChildren": False})
                        seen_communities.append(child_community[i])
        if len(children) == 1:
            try: 
                return children[0]['children']
            except:
                return []
        else:
            return children
        
def recursiveNaming(tree):
        i = tree['indices']
        base = H[i[0]]
        color = sum(B)[i[0]]
        if len(i) > 1:
            for ind in i[1:]:
                base = np.add(H[ind], base)
                color = color + sum(B)[ind]
        print color
        tree['color'] = color
        tree['name'] = " ".join([tfidf_feature_names[j] for j in base.argsort()[:-n_top_words - 1:-1]])
        if tree['hasChildren']:
            for child in tree['children']:
                    recursiveNaming(child)

In [74]:
tv = greedy_TV_build(thresh_vals(100),2)
# visualize topic tree
# for i in tv:
#     ccB = connected_components(build_wgraph(i))[1]
#     print ccB
size = len(tv)

In [93]:
flare = {"color" : sum(sum(B)), "name" : "" , "children" : populateTree(0, 0)}
for child in flare['children']:
    recursiveNaming(child)


(0.15296248266397466, 0)
(0.15296248266397466, 0)
(0.15296248266397466, 0)
(0.15296248266397466, 0)
(0.15296248266397466, 0)
(0.15296248266397466, 0)
(0.14969462688243218, 2)
(0.14969462688243218, 2)
(0.14969462688243218, 2)
(0.14969462688243218, 2)
(0.09976304123935989, 9)
(0.09976304123935989, 9)
(0.10705001947149878, 12)
(0.10705001947149878, 12)
(0.01039877412848786, 17)
(0.01039877412848786, 17)
(0.09976304123935989, 9)
(0.09976304123935989, 9)
(0.14969462688243218, 2)
(0.0074846184775288517, 7)
(0.0074846184775288517, 7)
(0.0074846184775288517, 7)
(0.0074846184775288517, 7)
(0.0074846184775288517, 7)
(0.14969462688243218, 2)
(0.15296248266397466, 0)
(0.15296248266397466, 0)
(0.016294406995603455, 8)
(0.016294406995603455, 8)
(0.016294406995603455, 8)
(0.016294406995603455, 8)
(0.016294406995603455, 8)
(0.016294406995603455, 8)
(0.016294406995603455, 8)
(0.016294406995603455, 8)
(0.15296248266397466, 0)
(0.0785604752736421, 5)
(0.0785604752736421, 5)
(0.0785604752736421, 5)
(0.078

In [94]:
with open('ss-demo.json', 'w') as outfile:
    json.dump(flare, outfile)

In [86]:
sum(sum(B))

1.754900012045385

In [87]:
255/1.754900012045385

145.30742392712756